# Import Libs

In [157]:
import re
import os

# Extraindo Personagens

## Descobrindo nomes arquivos de entrada

In [158]:
def list_path_temporadas() -> list:
    dir_dados = 'data'
    diretorios = os.listdir(dir_dados)
    diretorios_temporadas = [f'{dir_dados}/{f}' for f in diretorios if os.path.isdir(f'{dir_dados}/{f}')]
    return diretorios_temporadas

In [159]:
def list_path_epi(path_temporada) -> list:
    diretorios = os.listdir(path_temporada)
    path_episodios = [f'{path_temporada}/{f}' for f in diretorios if os.path.isfile(f'{path_temporada}/{f}')]
    return path_episodios

# Extrair Personagens

In [160]:
def regex_nome_personagem(string):
    return re.search('^[^"\[]*?:', string.upper())

In [161]:
def tratamento_nome_personagem(personagem_str):
    p = re.compile('((\(.*\))*)(\s?):')
    personagem_str = p.sub('', personagem_str) 
    p = re.compile('\s$')
    personagem_str = p.sub('', personagem_str)
    
    if 'YOUNG ' in personagem_str:
        return personagem_str.replace('YOUNG ', '')
    elif ' VOICE' in personagem_str:
        return personagem_str.replace('\'S VOICE', '')    
    
    return personagem_str

In [162]:
def contem_digito(personagem_str):
    return any(char.isdigit() for char in personagem_str)

In [163]:
def extrair_personagens_episodio(path_arquivo, personagens, personagens_figurantes):
    with open(path_arquivo, 'r') as file:
        for line in file:
            personagem = regex_nome_personagem(line)

            if personagem:
                personagem_str = tratamento_nome_personagem(personagem.group(0))
                
                contains_digit = any(char.isdigit() for char in personagem_str)
                
                if contains_digit:
                    p = re.compile('(\s?).(\d)')
                    personagem_figurante = p.sub('', personagem_str)
                    personagens_figurantes.add(personagem_figurante)
                else:
                    x = re.search('((^ALL\s)|(^ALL$)|(^AN?\s))', personagem_str)
                    if not x:
                        personagens.add(personagem_str)

In [164]:
def personagem_invalido(personagem_str):
    return ',' in personagem_str or '.' in personagem_str or 'CUT TO' in personagem_str or 'VOICE ' in personagem_str

In [165]:
# def tratamento_nome_personagem(personagem_str):
#     if 'YOUNG ' in personagem_str:
#         return personagem_str.replace('YOUNG ', '')
#     elif ' VOICE' in personagem_str:
#         return personagem_str.replace(' VOICE', '')
#     return personagem_str

In [166]:
def extrair_personagens():
    personagens = set()
    
    personagens_figurantes = set()
    
    for path_temporada in list_path_temporadas():
        for path_episodio in list_path_epi(path_temporada):
            extrair_personagens_episodio(path_episodio, personagens, personagens_figurantes)
            
    personagens_principais = set()
    for personagem in personagens:
        if personagem_invalido(personagem):
            continue
        
        # if 'YOUNG ' in personagem:
        #     personagens_principais.add(personagem.replace('YOUNG ', ''))
        #     continue
        # elif ' VOICE' in personagem:
        #     personagens_principais.add(personagem.replace(' VOICE', ''))
        #     continue       
        
        # personagem = tratamento_nome_personagem(personagem)
             
        if personagem not in personagens_figurantes:
            personagens_principais.add(personagem)    
             
    return personagens_principais

In [167]:
personagens = extrair_personagens()

# Divisão Cenas

In [168]:
def linha_atual_dialogo(personagem_regex):
    return False if not personagem_regex else True
    

In [169]:
def personagens_na_mesma_cena(dialogos, personagens):
    personagens_cena = set()
    
    for dialogo in dialogos:
        nome_personagem = regex_nome_personagem(dialogo).group(0)
        nome_personagem = tratamento_nome_personagem(nome_personagem)
        
        if personagem_invalido(nome_personagem):
            continue
        
        if nome_personagem in personagens:    
            personagens_cena.add(nome_personagem)
        
    return personagens_cena

In [170]:
def nova_cena(ultima_linha_dialogo, linha_atual_contem_dialogo, dialogo_iniciado_capitulo, line):
    if 'scene' in line:
        return True
    if '- - -' in line:
        return True
    if 'EXT.' in line: 
        return True
    if 'CUT TO' in line:
        return True
    if not ultima_linha_dialogo and dialogo_iniciado_capitulo:
        if linha_atual_contem_dialogo:
            return True
    return False
            

In [171]:
def identificacao_cenas(personagens):
    personagens_cenas_serie = []
    for path_temporada in list_path_temporadas():
        for path_episodio in list_path_epi(path_temporada):
            
            with open(path_episodio, 'r') as file:
                dialogos = []
                ultima_linha_dialogo = False
                gerar_quebra_cenario = False
                dialogo_iniciado_capitulo = False
                for line in file:
                    
                    if line == '\n':
                        continue
                    
                    personagem_regex = regex_nome_personagem(line)

                    linha_atual_contem_dialogo = linha_atual_dialogo(personagem_regex)                   

                    
                    if personagem_regex:
                        dialogo_iniciado_capitulo = True
                        dialogos.append(line)
                    elif nova_cena(ultima_linha_dialogo, linha_atual_contem_dialogo, dialogo_iniciado_capitulo, line):
                        personagens_cena = personagens_na_mesma_cena(dialogos, personagens)
                        if len(personagens_cena) > 1:
                            personagens_cenas_serie.append(personagens_cena)
                        dialogos = []
                    
                    ultima_linha_dialogo = linha_atual_contem_dialogo         
                        
    return personagens_cenas_serie                    

In [172]:
identificacao_cenas(personagens)

{'ROOSE', 'WILLEM', 'LORD OF BONES', 'ROSLIN', 'WOMEN', 'KING’S LANDING GUARD', 'MILITANT', 'BRAN STARK', 'SANSA', 'FREY GUARD', 'SEPTA UNELLA', 'ROBERT', 'BRAN', 'COLEN', 'BLONDE PROSTITUTE', 'MERYN TRANT', 'MYRCELLA', 'EDDARD STARK', 'DRIVER', 'YARA', 'BOY', 'LORD VARYS', 'CAPTAIN', 'SEVERAL STARK BANNERMEN', 'DERRYK', 'EURION', 'DAARIO NAHARIS', "OLLY'S MOTHER", 'OLENNA', 'HEAD PROSTITUTE', 'NIGHT’S WATCH BROTHER', 'CRAYAH', 'CERSEI BARATHEON', 'ARTHUR', 'NED & ALYS', 'KHAL DROGO', 'DAENERYS STORMBORN', 'JONOS BRACKEN', 'ELARIA', 'CAMELLO', 'QHORIN', 'MELESSA', 'DOLOROUD EDD', 'MALKO', 'RENNICK', 'MERYN', 'SLAVES', 'SEPTA', 'WAIF', 'SER DONTOS', 'DEANERYS TARGARIAN', 'RANGER', 'FREY MAN', 'TYRON', 'WILL', 'TANNER', 'BRONN', 'HARRAG', 'STANNIS', 'LEM', 'THE HOUND', 'YOREN', 'RAMSAY', 'LOLLYS STOKEWORTH', 'LYANNA MORMONT', 'LADY CRANE', 'TYWIN LANNISTER', 'MARTYN', 'GUARD CAPTAIN', 'LOMMY GREENHANDS', 'SEPTON', 'JANOS', 'LADY ANYA', 'RAKHARO', 'DANAERYS', 'OBARA', 'PYCELLE', 'SER JORA